In [2]:
import pandas as pd 
import numpy as np 
import re
import Actu_Colums as Ac

In [3]:
def split_column(df, column_name,colum1,colum2):
    # Define una función para dividir una cadena cuando pasa de un número a un "-", "_", " " o una letra
    def split_string(s):
        return re.split(r'(?<=[0-9])(?=[-_ A-Za-z])', s)
    df1= df.copy()
    # Aplica la función a la columna especificada y la divide en dos columnas
    df1[[f'{colum1}', f'{colum2}']] = df1[column_name].apply(split_string).apply(pd.Series)

    # Elimina el delimitador de la segunda columna
    df1[f'{colum2}'] = df1[f'{colum2}'].str.lstrip('-_ ')
    df1.drop(columns=[f'{column_name}'],inplace=True)

    # Devuelve el DataFrame modificado
    return df1


def compact_rows(df:pd.DataFrame, columns:list, delimiter:str ='/'):
    for col in df.columns:
        df[col] = df[col].astype(str)
    
    # Usar 'join' como función de agregación para concatenar los valores
    agg_func = {col: lambda x: '/'.join(x.unique()) for col in df.columns if col not in  columns}
    
    # Agrupar por 'OC Posición' y aplicar la función de agregación
    df1 = df.groupby(columns).agg(agg_func).reset_index()
    return df1
def combine_and_rename(df1, combine_cols, new_names):
    df = df1.copy()
    for cols in combine_cols:
        df[cols[0]] = df[cols[0]].combine_first(df[cols[1]])
    df.rename(columns=new_names, inplace=True)
    df.drop(columns=[col for sublist in combine_cols for col in sublist[1:]], inplace=True)
    return df

## Capi 3.0

In [4]:
CapiV3 = pd.read_excel('D:/Excels/06.Reporte de OEP Dir Tecnologia Junio 24.xlsx',sheet_name='OEP 2024-06')

C:\Users\C26764\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [5]:
CapiV3.info()
CapiV3_mod = CapiV3.dropna(how='all')
new_header = CapiV3_mod.iloc[1] # tomar la primera fila para el encabezado
CapiV3_mod = CapiV3_mod[2:] # tomar los datos menos la fila del encabezado
CapiV3_mod.columns = new_header # establecer la fila 
CapiV3_mod = CapiV3_mod[CapiV3_mod.Gerente == 'Eduardo Iberico']
CapiV3_mod = CapiV3_mod[CapiV3_mod['SUSTENTO DIREC'] != 'IMPUTACION ERRONEA']
#CapiV3_mod = CapiV3_mod[CapiV3_mod['Comentario Soporte'] != 'NO CAPITALIZAR']
#CapiV3_mod_EI['COMPROMISO CAPITALIZACION'] = CapiV3_mod_EI['COMPROMISO CAPITALIZACION'].str.replace('00:00:00','')
CapiV3_mod['COMPROMISO CAPITALIZACION'] = pd.to_datetime(CapiV3_mod['COMPROMISO CAPITALIZACION'])
CapiV3_mod = CapiV3_mod.T.drop_duplicates(keep='last').T
duplicated_columns = CapiV3_mod.columns[CapiV3_mod.columns.duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31608 entries, 0 to 31607
Columns: 132 entries, Unnamed: 0 to Unnamed: 131
dtypes: object(132)
memory usage: 31.8+ MB


In [6]:
CapiV3_mod.columns.tolist()

['ITEM',
 'Analista',
 'PATRIMONIO',
 'Doc. CO',
 'Apunte',
 'Llave',
 'OBRAS',
 'PERIODO',
 'MES',
 'AÑO CAPEX',
 'F. ALTA AGRUPADO',
 'F.Alta',
 'Fe. Corte',
 'Dias transcurridos',
 'Parametro Dias',
 'OKR',
 'MES OKR',
 'Parametro Proy Regulares_Revision',
 'Parametro Qlik (Clasifi 3)',
 'Parametros PPT',
 'Parametro Largo Plazo',
 'Tipo de validación',
 'Proyecto',
 'Elemento PEP',
 'Clase',
 'Cuenta CAP',
 'Activo Fijo',
 'SN°',
 'ID Nat.',
 'Denominación Activo Fijo',
 'Status Site',
 'Tpo Mat.',
 'Material',
 'Texto de Material',
 'Acreedor',
 'Nombre',
 '  Cant. Regi',
 'Val. Mon. Obj.',
 'Importe en Miles Soles',
 'TC',
 'Importe en Miles Dolares',
 'F. Cont.',
 'Cls. Doc',
 'N° D.Ref',
 'Doc.Anul',
 'Anulado',
 'Ref.Anul',
 'Usuario',
 'Documento compras',
 'Posición',
 'OC+POS',
 '   % EA',
 '% Acept',
 ' % Fact',
 'Fin. Pos.',
 'Fin. Mat.',
 'Status EA',
 'Status Acept',
 'CeCo Sol',
 'CeCo Resp.',
 'ID TipRed',
 'ID TipProy',
 'ID R.Inv.',
 'Folio / Re',
 'Aprobador',
 'St

In [7]:
Capi_agg_0 = pd.pivot_table(CapiV3_mod, 
                          values='Importe en Miles Dolares', 
                          index=['Denominación Activo Fijo','Sustento/Solicitud PAP','Status Solicitud PAP','OC+POS',
                                 'COMPROMISO CAPITALIZACION','Nombre'], 
                          aggfunc='sum').reset_index()


Capi_agg_0.replace('#', pd.NA, inplace=True)
Capi_agg_0 = Capi_agg_0.groupby('Denominación Activo Fijo').apply(lambda group: group.ffill().bfill())

# Resetear el índice para aplanar el dataframe después de la operación groupby
Capi_agg_0.reset_index(drop=True, inplace=True)

# Convertir la columna 'Nombre Importe en Miles Dolares' a valores numéricos
Capi_agg_0['Importe en Miles Dolares'] = pd.to_numeric(Capi_agg_0['Importe en Miles Dolares'], errors='coerce')

Capi_agg_0 = Capi_agg_0.sort_values(by='Denominación Activo Fijo')
Capi_agg_0_NF = Capi_agg_0[Capi_agg_0['Status Solicitud PAP'] != 'Finalizado'].copy()
Capi_agg_0_NF.rename(columns={'Denominación Activo Fijo':'Site'},inplace=True)



In [8]:
Capi_agg_0_NF[Capi_agg_0_NF['Site'] == 'TP6276-TALARA']

3,Site,Sustento/Solicitud PAP,Status Solicitud PAP,OC+POS,COMPROMISO CAPITALIZACION,Nombre,Importe en Miles Dolares
827,TP6276-TALARA,58499,Pre registro,4500660538-890,2024-07-31,MEG PERU SAC,2.114415


In [9]:
Capi_agg_0_NF[Capi_agg_0_NF['COMPROMISO CAPITALIZACION'] <= '2024-07-30'].sort_values(by='Importe en Miles Dolares',ascending=False)

3,Site,Sustento/Solicitud PAP,Status Solicitud PAP,OC+POS,COMPROMISO CAPITALIZACION,Nombre,Importe en Miles Dolares
661,TM5579-EL CHIDO,50903,Observado,4500627223-40,2024-06-30,CJ TELECOM S.A.C.,1.815129
664,TM5579-EL CHIDO,58362,Pre registro,4500660863-40,2024-06-30,CJ TELECOM S.A.C.,0.461035


In [10]:
fecha_inicio = '2024-07-01'
fecha_fin = '2024-08-30'

Capi_agg_0_NF_Julio = Capi_agg_0_NF[(Capi_agg_0_NF['COMPROMISO CAPITALIZACION'] >= fecha_inicio) & 
                                       (Capi_agg_0_NF['COMPROMISO CAPITALIZACION'] <= fecha_fin)]

Capi_agg_0_NF_Julio.sort_values(by='Importe en Miles Dolares', ascending=False)


3,Site,Sustento/Solicitud PAP,Status Solicitud PAP,OC+POS,COMPROMISO CAPITALIZACION,Nombre,Importe en Miles Dolares
525,TL2009-NUEVA_ARICA,58897,Pre registro,4500639731-40,2024-07-31,CJ TELECOM S.A.C.,57.660219
538,TL2959-SAN_AGUSTIN,60943,Visita ejecutada,4500696049-10,2024-07-31,CJ TELECOM S.A.C.,49.038590
523,TL2009-NUEVA_ARICA,58883,FAC,4500639731-30,2024-07-31,CJ TELECOM S.A.C.,32.586635
564,TL5573-NAT_CHOSICA,<NA>,NaN,NaN,2024-07-31,NaN,16.335374
113,TA6453_NAT_HUMANHUANCO,58523,Pre registro,4500660255-40,2024-07-31,MEG PERU SAC,10.631483
7,TA0342 NAT_JULIO_C_TELLO,58516,Pre registro,4500659991-40,2024-07-31,MEG PERU SAC,10.347928
347,TJ4155-NAT_PUENTE_VIRU_P1,62070,Pre registro,4500660642-10,2024-07-31,CJ TELECOM S.A.C.,10.240513
775,TP61427-NAT_PAITA_P2,62148,Pre registro,4500642589-60,2024-07-31,CJ TELECOM S.A.C.,9.635913
20,TA2488-HUALLANCA_2,61926,Pre registro,4500666390-10,2024-07-31,CICSA PERU S.A.C.,9.284597
777,TP6147-NAT_SUNCHINE,58919,Pre registro,4500663977-10,2024-07-31,CJ TELECOM S.A.C.,9.247032


In [11]:
Capi_agg_0_NF_Julio['Importe en Miles Dolares'].sum()

339.5109382329945

In [12]:
lista = ['FAC', 'Aprobaciones FAC','Visita ejecutada','Observado']
Capi_agg_0_NF_Julio['Status Solicitud PAP'].value_counts()

Status Solicitud PAP
Pre registro        30
Visita ejecutada     4
Aprobaciones FAC     3
Observado            3
FAC                  2
Registrado           2
Name: count, dtype: int64

In [13]:
Capi_agg_0_NF_Julio = Capi_agg_0_NF_Julio[~Capi_agg_0_NF_Julio['Status Solicitud PAP'].isin(lista)] ## Compruebo monto

In [14]:
Capi_agg_0_NF_Julio_sort = Capi_agg_0_NF_Julio.groupby(['Site','Sustento/Solicitud PAP','Status Solicitud PAP','OC+POS','COMPROMISO CAPITALIZACION','Nombre'])\
                                    ['Importe en Miles Dolares'].sum().reset_index(name='Total del Proyecto en Miles de Dolares')
Capi_agg_0_NF_Julio_sort                                                                                                                                                        

,Site,Sustento/Solicitud PAP,Status Solicitud PAP,OC+POS,COMPROMISO CAPITALIZACION,Nombre,Total del Proyecto en Miles de Dolares
0,TA0342 NAT_JULIO_C_TELLO,58516,Pre registro,4500659991-40,2024-07-31,MEG PERU SAC,10.347928
1,TA0342 NAT_JULIO_C_TELLO,58518,Pre registro,4500659991-60,2024-07-31,MEG PERU SAC,3.622231
2,TA0342 NAT_JULIO_C_TELLO,58522,Pre registro,4500659991-50,2024-07-31,MEG PERU SAC,3.101381
3,TA2488-HUALLANCA_2,61926,Pre registro,4500666390-10,2024-07-31,CICSA PERU S.A.C.,9.284597
4,TA5990-MINERA_SANTA_LUISA,59088,Pre registro,4500679881-20,2024-07-31,MEG PERU SAC,3.952492
5,TA6453_NAT_HUMANHUANCO,58515,Pre registro,4500660255-60,2024-07-31,MEG PERU SAC,4.691165
6,TA6453_NAT_HUMANHUANCO,58520,Pre registro,4500660255-50,2024-07-31,MEG PERU SAC,3.622622
7,TA6453_NAT_HUMANHUANCO,58523,Pre registro,4500660255-40,2024-07-31,MEG PERU SAC,10.631483
8,TJ4155-NAT_PUENTE_VIRU_P1,62070,Pre registro,4500660642-10,2024-07-31,CJ TELECOM S.A.C.,10.240513
9,TJ4155-NAT_PUENTE_VIRU_P1,62071,Pre registro,4500660642-20,2024-07-31,CJ TELECOM S.A.C.,3.716841


## Preparo para el merge

In [15]:
Capi_agg_0_NF_Julio_split = split_column(Capi_agg_0_NF_Julio_sort,'Site','ID SITE','SITE')

Capi_agg_0_NF_Julio_split.SITE = Capi_agg_0_NF_Julio_split.SITE.str.title().str.replace('_',' ')

In [16]:
prepa_f = pd.read_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Reporte Prepasivo.csv')

prepa2merge = prepa_f[['Site','Responsable3','Contratista','Antiguamiento Proyectado','N° Pap','Estado','Comentario']].drop_duplicates(subset=['N° Pap','Site'])
prepa2merge.rename(columns={'Site':'SITE','N° Pap':'Sustento/Solicitud PAP'},inplace=True)

In [17]:
## agg de rows pre merge 
prepa_agg2merge1 = compact_rows(prepa2merge,['SITE'],'/')
prepa_agg2merge0 = compact_rows(prepa2merge,['Sustento/Solicitud PAP'],'/')

In [18]:
prepa_agg2merge0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sustento/Solicitud PAP    171 non-null    object
 1   SITE                      171 non-null    object
 2   Responsable3              171 non-null    object
 3   Contratista               171 non-null    object
 4   Antiguamiento Proyectado  171 non-null    object
 5   Estado                    171 non-null    object
 6   Comentario                171 non-null    object
dtypes: object(7)
memory usage: 9.5+ KB


In [19]:
columns = prepa_agg2merge0.columns.tolist()
columns.remove('SITE')

In [20]:
Capi_agg_0_NF_Julio_split['Sustento/Solicitud PAP'] = Capi_agg_0_NF_Julio_split['Sustento/Solicitud PAP'].astype(str)
prepa_agg2merge0['Sustento/Solicitud PAP'] = prepa_agg2merge0['Sustento/Solicitud PAP'].astype(str)

capi_merged_PAP = pd.merge(Capi_agg_0_NF_Julio_split,prepa_agg2merge0[columns],how='left',on=['Sustento/Solicitud PAP'])

In [21]:
capi_merged_PAP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 13 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Sustento/Solicitud PAP                  29 non-null     object        
 1   Status Solicitud PAP                    29 non-null     object        
 2   OC+POS                                  29 non-null     object        
 3   COMPROMISO CAPITALIZACION               29 non-null     datetime64[ns]
 4   Nombre                                  29 non-null     object        
 5   Total del Proyecto en Miles de Dolares  29 non-null     float64       
 6   ID SITE                                 29 non-null     object        
 7   SITE                                    29 non-null     object        
 8   Responsable3                            29 non-null     object        
 9   Contratista                             29 non-null     

In [22]:
capi_merged_PAP['Sustento/Solicitud PAP'] = capi_merged_PAP['Sustento/Solicitud PAP'].replace('#',np.nan)

In [23]:
columns1 = prepa_agg2merge1.columns.tolist()
#columns1.remove('Sustento/Solicitud PAP')

In [24]:
capi_merged = pd.merge(capi_merged_PAP,prepa_agg2merge1[columns1],how='left',on=['SITE'])

In [25]:
capi_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 19 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Sustento/Solicitud PAP_x                29 non-null     object        
 1   Status Solicitud PAP                    29 non-null     object        
 2   OC+POS                                  29 non-null     object        
 3   COMPROMISO CAPITALIZACION               29 non-null     datetime64[ns]
 4   Nombre                                  29 non-null     object        
 5   Total del Proyecto en Miles de Dolares  29 non-null     float64       
 6   ID SITE                                 29 non-null     object        
 7   SITE                                    29 non-null     object        
 8   Responsable3_x                          29 non-null     object        
 9   Contratista_x                           29 non-null     

In [26]:
capi_merged = combine_and_rename(capi_merged,[('Responsable3_x', 'Responsable3_y'),('Contratista_x','Contratista_y'),
                                             ('Antiguamiento Proyectado_x','Antiguamiento Proyectado_y'),('Estado_x','Estado_y')
                                              ,('Comentario_x','Comentario_y'),('Sustento/Solicitud PAP_x','Sustento/Solicitud PAP_y')],
                                                 {'Responsable3_x':'Responsable3','Contratista_x':'Contratista','Antiguamiento Proyectado_x':'Antiguamiento Proyectado',
                                                 'Estado_x':'Estado','Comentario_x':'Comentario','Sustento/Solicitud PAP_x':'Sustento/Solicitud PAP'})
capi_merged.drop(columns=['Status Solicitud PAP'],inplace=True)

In [27]:
capi_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 12 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Sustento/Solicitud PAP                  29 non-null     object        
 1   OC+POS                                  29 non-null     object        
 2   COMPROMISO CAPITALIZACION               29 non-null     datetime64[ns]
 3   Nombre                                  29 non-null     object        
 4   Total del Proyecto en Miles de Dolares  29 non-null     float64       
 5   ID SITE                                 29 non-null     object        
 6   SITE                                    29 non-null     object        
 7   Responsable3                            29 non-null     object        
 8   Contratista                             29 non-null     object        
 9   Antiguamiento Proyectado                29 non-null     

In [28]:
capi_merged['Total del Proyecto en Miles de Dolares'].sum()

198.1531378681261

In [29]:
capi_merged.sort_values(by='Total del Proyecto en Miles de Dolares',ascending=False)

,Sustento/Solicitud PAP,OC+POS,COMPROMISO CAPITALIZACION,Nombre,Total del Proyecto en Miles de Dolares,ID SITE,SITE,Responsable3,Contratista,Antiguamiento Proyectado,Estado,Comentario
17,58897,4500639731-40,2024-07-31,CJ TELECOM S.A.C.,58.886177,TL2009,Nueva Arica,JHORDAN,Cj Telecom S.A.C.,ON AIR,Pre registro,Falta todo en el PAP
23,62148,4500642589-60,2024-07-31,CJ TELECOM S.A.C.,12.379901,TP61427,Nat Paita P2,DEMETRIO,Cj Telecom S.A.C.,ON AIR,Pre registro,Pendiente Liquid Claro
7,58523,4500660255-40,2024-07-31,MEG PERU SAC,10.631483,TA6453,Nat Humanhuanco,DEMETRIO,Meg Peru S.A.C.,CW,Pre registro,Por ejecutar
0,58516,4500659991-40,2024-07-31,MEG PERU SAC,10.347928,TA0342,Nat Julio C Tello,DEMETRIO,Meg Peru S.A.C.,TI,Pre registro,Por ejecutar/2024-08-26/EN DEFINICION DE SAQ ...
8,62070,4500660642-10,2024-07-31,CJ TELECOM S.A.C.,10.240513,TJ4155,Nat Puente Viru P1,DEMETRIO,Cj Telecom S.A.C.,ACTIVO,Pre registro,Activo
3,61926,4500666390-10,2024-07-31,CICSA PERU S.A.C.,9.284597,TA2488,Huallanca 2,JHORDAN,Cicsa Peru S.A.C.,ON AIR,Pre registro,Pend GIS
24,58919,4500663977-10,2024-07-31,CJ TELECOM S.A.C.,9.247032,TP6147,Nat Sunchine,DEMETRIO,Cj Telecom S.A.C.,ON AIR,Registrar Pendientes,PEND GIS/ATP
11,58889,4500661619-10,2024-07-31,CJ TELECOM S.A.C.,8.686878,TJ49484,Nat Llacuabamba P3,DEMETRIO,Cj Telecom S.A.C.,OH,Pre registro,"EJECUTADO/2024-07-15/CW OK, Pendiente energia ..."
13,58894,4500661619-30,2024-07-31,CJ TELECOM S.A.C.,7.239479,TJ49484,Nat Llacuabamba P3,DEMETRIO,Cj Telecom S.A.C.,OH,Pre registro,Por ejecutar/2024-07-15/nan
28,56701,4500660086-50,2024-07-31,CICSA PERU S.A.C.,5.270305,TP6285,Las Lomas,ANGGIE,Cicsa Peru S.A.C.,CW/TI,Finalizado,Ejecutado/-/Devol Equip D.


In [30]:
Ac.Excel_format(capi_merged,r'D:\Capi\Capi.xlsx',['Status Solicitud PAP','Nombre']) 


In [29]:
capi_merged['Total del Proyecto en Miles de Dolares'].sum()

622.6977326035964

In [44]:
Capi_agg['Importe en Miles Dolares'].sum()

3596.042444618196

## Capilatizar 2.0



In [ ]:
capiV2 = pd.read_excel('D:/Excels/Capi2.0.xlsx',sheet_name='detalle')

In [39]:
capiV2 = capiV2.dropna(how='all')
capiV2 = capiV2.dropna(axis=1,how='all')
new_header = capiV2.iloc[1] # tomar la primera fila para el encabezado
capiV2 = capiV2[2:] # tomar los datos menos la fila del encabezado
capiV2.columns = new_header # establecer la fila 
capiV2_EI = capiV2[capiV2.Gerente == 'Eduardo Iberico']

In [40]:
capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'] = pd.to_datetime(capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'],format = '%Y-%m-%d')
capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'] = capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'].dt.strftime('%m-%Y')
capiV2_EI['**Fsct de Capitalizacion en Letras Qlik']= pd.to_datetime(capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'])

C:\Users\C26764\AppData\Local\Temp\ipykernel_28212\4196960423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'] = pd.to_datetime(capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'],format = '%Y-%m-%d')
C:\Users\C26764\AppData\Local\Temp\ipykernel_28212\4196960423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'] = capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'].dt.strftime('%m-%Y')
C:\Users\C2

In [41]:
capi_filtred = capiV2_EI[capiV2_EI['**Fsct de Capitalizacion en Letras Qlik'].dt.month <= 4].iloc[:,5:35]


In [95]:
CapiN_con = capi_filtred[capi_filtred['Status AL'] != 'CONCLUIDO']
CAPITALIZACION_Pend = CapiN_con[CapiN_con['Status Solicitud PAP'] !='Finalizado']

In [96]:
CAPITALIZACION_Pend = CAPITALIZACION_Pend.groupby(['Parametro Proy Regulares_1','Denominación Activo Fijo','Texto de Material',
                                                   'Sustento/Solicitud PAP','Status Solicitud PAP'])['Importe en Miles Dolares'].sum().reset_index()

In [97]:
CAPITALIZACION_Pend['Sustento/Solicitud PAP'] = CAPITALIZACION_Pend['Sustento/Solicitud PAP'].replace('#','Sin Pap')
CAPITALIZACION_Pend['Status Solicitud PAP'] = CAPITALIZACION_Pend['Status Solicitud PAP'].replace('#','Sin Pap')
CAPITALIZACION_Pend = CAPITALIZACION_Pend.sort_values(by='Importe en Miles Dolares',ascending=False)
CAPITALIZACION_Pend.rename(columns=({'Sustento/Solicitud PAP':'PAP'}),inplace=True)

In [70]:
df_2_merge = PRE_all_act[['PAP','COMENTARIO','ESTADO','RESPONSABLE3','NOMPROVEEDOR']]

In [71]:
def process_PAP(value):
    if any(map(str.isdigit, str(value))):
        value_float = int(value)
        value_str = str(value_float)
        # Rest of your code

        return value_str
    else:
        return value

In [72]:
df_2_merge['PAP'] = df_2_merge['PAP'].apply(process_PAP)

C:\Users\C26764\AppData\Local\Temp\ipykernel_28212\3201755170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_merge['PAP'] = df_2_merge['PAP'].apply(process_PAP)


In [74]:
df_2_merge_NN = df_2_merge[df_2_merge['PAP'] != 'Sin pap']
df_2_merge_NN

,PAP,COMENTARIO,ESTADO,RESPONSABLE3,NOMPROVEEDOR
0,51948,Falta Gis.Liqui,Solic FAC,JHORDAN,CJ TELECOM S.A.C.
1,55489,"Por cerrar, pos separadas",Finalizado,JENNY,HUAWEI DEL PERU SAC
3,52531,Falta GIS,Solic FAC,JHORDAN,CJ TELECOM S.A.C.
4,53502,Valid GIS,Solic FAC,DANNER,CICSA PERU S.A.C.
5,53502,Valid GIS,Solic FAC,DANNER,CICSA PERU S.A.C.
...,...,...,...,...,...
2111,56093,Pend GIS/CF,Solic FAC,ANGGIE,MEG PERU SAC
2952,58206,NaN,Rechazado,ANGGIE,CICSA PERU S.A.C.
410,58531,Alta de suministro,Rechazado,DEMETRIO,MEG PERU SAC
1122,58913,Duplicado,Rechazado,JHORDAN,CJ TELECOM S.A.C.


In [100]:
CAPITALIZACION_Pend.PAP = CAPITALIZACION_Pend.PAP.astype(str)

In [102]:
CAPITALIZACION_Pend[CAPITALIZACION_Pend['PAP'] == '52531']

,Parametro Proy Regulares_1,Denominación Activo Fijo,Texto de Material,PAP,Status Solicitud PAP,Importe en Miles Dolares
5,De 6 Meses a 1 Año,TA3878-CHIPTA,OBRA CIVIL SITIOS TECNICOS,52531,FAC,87.719261


In [90]:
df_2_merge_NN

,PAP,COMENTARIO,ESTADO,RESPONSABLE3,NOMPROVEEDOR
0,51948,Falta Gis.Liqui,Solic FAC,JHORDAN,CJ TELECOM S.A.C.
1,55489,"Por cerrar, pos separadas",Finalizado,JENNY,HUAWEI DEL PERU SAC
3,52531,Falta GIS,Solic FAC,JHORDAN,CJ TELECOM S.A.C.
4,53502,Valid GIS,Solic FAC,DANNER,CICSA PERU S.A.C.
5,53502,Valid GIS,Solic FAC,DANNER,CICSA PERU S.A.C.
...,...,...,...,...,...
2111,56093,Pend GIS/CF,Solic FAC,ANGGIE,MEG PERU SAC
2952,58206,NaN,Rechazado,ANGGIE,CICSA PERU S.A.C.
410,58531,Alta de suministro,Rechazado,DEMETRIO,MEG PERU SAC
1122,58913,Duplicado,Rechazado,JHORDAN,CJ TELECOM S.A.C.


In [109]:
PAP_S.PAP = PAP_S.PAP.astype(str)

In [112]:
PAP_S

,PAP,ESTADO_PAP,Acción,RESPONSABLE_PAP,ID Site,SITE,ANTIGUAMIENTO_PAP,F.Creación,new_column_2,OC Posición
44,60736,Pre registro,Pasar a Registrado,CJ TELECOM S.A.C.,TJ4233,PARIAMARCA,2.0,13/04/2024 2:04:52 PM,None,4500698717:20
44,60736,Pre registro,Pasar a Registrado,CJ TELECOM S.A.C.,TJ4233,PARIAMARCA,2.0,13/04/2024 2:04:52 PM,None,4500698717:10
46,60734,Pre registro,Pasar a Registrado,COMUNICACION FUTURA SOCIEDAD ANONIM,LI1017,PARDO_BARREDA,2.0,13/04/2024 12:04:21 PM,None,4500688022:60
46,60734,Pre registro,Pasar a Registrado,COMUNICACION FUTURA SOCIEDAD ANONIM,LI1017,PARDO_BARREDA,2.0,13/04/2024 12:04:21 PM,None,4500688022:50
151,60629,Pre registro,Pasar a Registrado,CICSA PERU S.A.C.,TC3526,SANTA_CRUZ_CENTRO,6.0,09/04/2024 5:23:51 PM,None,4500687406:10
...,...,...,...,...,...,...,...,...,...,...
51947,8815,Rechazado,Generar nueva solicitud,SOLUCIONES TECNOLOGICAS LATINOAMERI,LJ2721,CALLE_REAL,227.0,20/02/2019 1:50:02 PM,NaN,4500258478:20
51948,8814,Rechazado,Generar nueva solicitud,SOLUCIONES TECNOLOGICAS LATINOAMERI,LJ2695,IRAYRAPATA,227.0,20/02/2019 1:45:37 PM,NaN,4500258454:20
53786,6974,FAC,Solicitar aprobaciones FAC,CICSA PERU S.A.C.,TC5830,JAEN,277.0,21/11/2018 1:47:58 PM,NaN,4500247504:750
58866,1894,FAC,Solicitar aprobaciones FAC,MEG PERU SAC,LI18481,SEDAPAL_TC_SC1,84.0,09/05/2018 8:51:09 PM,NaN,4500223695:10


In [110]:
df_merged_1 = pd.merge(CAPITALIZACION_Pend,df_2_merge_NN,on='PAP',how='left')
df_merged_fnl = pd.merge(df_merged_1,,on='PAP',how='left')